In [ ]:
import initialize

In [ ]:

# 1) create the optimization algorithm
# 2) decide on th


# x*1 + y* 2
# lambda x, y: x * 1 + y * 2
# Constaint = LTE(x=100, y=50) & GTE(x=0, y=0)
# genetic = Genetic()
# fitter.fit(func, constraint, x=20, y=30, const=['x'], penalty=-inf)
# fitter.step( state=state)
# 1) create the individual -> only x
# 2) populate
# 3) If not first
#    get children
#    cross over
#    elitism
# 3) mutate
# 4) assess
# 5) yield assessments

In [ ]:
from zenkai import tansaku

x = 20
y = 30
populator = tansaku.RepeatPopulator(10)


objective = tansaku.FuncObjective(
    lambda x, y: 2 * x + 2.5 * y 
)

# how to convert to float
individual = tansaku.Individual(x=20.0, y=30.0)
population = populator(individual)

mutator = tansaku.GaussianMutator(0.2, 0.0)
population = mutator(population)

population = tansaku.Population(
    x=population['x'].clamp(0, 30),
    y=population['y'].clamp(0, 20)
)



print(population['x'])

# # Add a "Lambda" constraint
# LC(['x', 't'], lambda x, t: 2 * x + 3 * t < 4)
# LC() + 

objective = tansaku.FuncObjective(lambda x, y: 3 * x + 5 * y, penalty=1000)
assessor = tansaku.ObjectivePopAssessor(objective, ['x', 'y'])

population = assessor(population)

divider = tansaku.FitnessProportionateDivider(6)
parents1, parents2 = divider(population)

print(parents1['x'], parents2['x'])
crossover = tansaku.SmoothCrossOverBreeder()
children =crossover(parents1, parents2)
print(children['x'])

elitism = tansaku.KBestElitism(2)
children2 = elitism(population, children)
print(children2['x'])


In [ ]:
import typing
from zenkai import tansaku
from zenkai.kaku import Assessment, State
import torch
from functools import partial

class GeneticAlgorithm(tansaku.Itadaki):

    def __init__(self) -> None:
        super().__init__()
        self.divider = tansaku.FitnessProportionateDivider(8)
        self.elitism = tansaku.KBestElitism(2)
        self.crossover = tansaku.SmoothCrossOverBreeder()
        self.mutator = tansaku.GaussianMutator(0.5, 0.0)
        self.populator = tansaku.RepeatPopulator(10)
        self.iterations = 100

    def optim_iter(self, objective: tansaku.Objective, state: State = None, **kwargs) -> typing.Iterator[Assessment]:
        
        state = state or State()
        my_state = state.mine((self, objective))
        assessor = tansaku.ObjectivePopAssessor(objective, ['x', 'y'])

        i = my_state.get_or_set('i', 0)
        if i == 0:
            individual = tansaku.Individual(**kwargs)
            population = self.populator(individual, state)
            population = self.mutator(population, state)
        else:
            population = my_state.get('population')

        while i < self.iterations:
            
            if i > 0:
                parents1, parents2 = self.divider(population, state)
                
                children = self.crossover(parents1, parents2, state)
                children = self.mutator(children, state)
                population = self.elitism(population, children, state)
            
            population = population.apply(partial(torch.clamp, min=0))

            population = assessor(population, state)
            my_state['population'] = population
            yield population.stack_assessments()
            i += 1


objective = tansaku.FuncObjective(
    lambda x, y: torch.abs(2 * x + 3 * y), constraint=tansaku.LTE(y=5, x=10) + tansaku.GTE(y=0, x=0), penalty=0, maximize=True
)


genetic_algorithm = GeneticAlgorithm()

for assessment in genetic_algorithm.optim_iter(objective, x=3.0, y=4.0):
    print(assessment)

    